# CS344 AI Final Report
By Henri Prudhomme

# Goal
The goal of my project has changed between iterations as I came upon different challenges, but the main idea always stayed the same. The goal was to write a chess AI in swi-prolog which implemented a minimax AI. Originally I had planned to implement Alpha-Beta pruning, as well as add expert rules to the heuristic evaluation of the chess board, but those had to be sidelined as time shrank and troubles increased.

# Background
I will be using prolog for my AI. Firstly, I chose prolog because it was one of the new languages introduced in the AI course, and I wanted to explore with the new language. Originally, I chose prolog because it allows very conventient expressions of if-then decisions for choosing a best move, however my belief that this would be helpful ended up being so WRONG, and I now would never recommend prolog for a chess AI implementation. Its doable, but very difficult. 

# Sources/Reference Material: 
There were no useful sources or reference material for my particular project. Because of this, all code written (with exception of a list manipulation algorithm cited at the end of this paper) comes directly from my head. 

# Implementation

## 1. Demo
The following describes the method for using the chess AI.

Usage:
- compute( Board, ColorToPlay).
  - Board is a list of lists of numbers representing square states
    - the elements use the following numbering system:
      a. 0 - empty
      b. 1 - white pawn
      c. 2 - white knight
      d. 3 - white bishop
      e. 4 - white rook
      f. 5 - white queen 
      g. 6 - white king
      h. 11 - black pawn
      i. 12 - black knight
      j. 13 - black bishop
      k. 14 - black rook
      l. 15 - black queen
      m. 16 - black king
  - ColorToPlay is an atom

Output: 
- List - a 2d representation of the solution chess board with the best score for the player specified
- Number - a number which represents the minimax evaluation of the solution board. A positive number is in favor of white, and a negative number is in favor of black. 
- Following the scoring system used by chess.com the pieces are valued by the following:

  - white pawn +1

  - white rook +5

  - white knight +3

  - white bishop +3

  - white queen +9

  - white king +1000

  - black pawn -1

  - black rook -5

  - black knight -3

  - black bishop -3

  - black queen -9

  - black king -1000

###Demo Run
(taken from an actual sample run)
```
?- compute([
[12, 13, 14,  0, 16, 14, 13, 12],
[11, 11, 11,  0, 11, 11, 11, 11],
[ 0,  0,  0,  0,  0,  0,  0,  0],
[ 0,  0,  0, 11,  0,  15,  0,  0],
[ 0,  0,  5,  0,  1,  0,  0,  0],
[ 0,  0,  0,  0,  0,  0,  0,  0],
[ 1,  1,  1,  1,  0,  1,  1,  1],
[ 2,  3,  4,  0,  6,  4,  3, 2]
], black).
[12,13,14,0,16,14,13,12]
[11,11,11,0,11,11,11,11]
[0,0,0,0,0,0,0,0]
[0,0,0,0,0,15,0,0]
[0,0,5,0,11,0,0,0]
[0,0,0,0,0,0,0,0]
[1,1,1,1,0,1,1,1]
[2,3,4,0,6,4,3,2]

-4
true .
```

### Explanation of Output
As one can see from this example, the chess AI decided that the black pawn taking the white pawn instead of taking the much higher valued white queen was the best move. At first, this may seem like a mistake, but as you will soon see, this is the best move. 

Looking 3 nodes deep, the AI was able to recognize an important fact. Taking the white queen instead of the white pawn , would leave white open to retaliate and take the black queen, resulting in a trade of pieces. The score of the situation just described would leave the score tied at 0.

By taking the pawn, black is able to get an advantage because white will no longer have a piece to attack black's queen, furthermore, the chess AI has determined that by taking this move, black will be guaranteed to be have a 4 point advantage over white within 3 moves. This is seen from the output score evaluation of the best move.

## 2. A Discussion of Programming Techniques in Prolog
In programming languages such as Java, C++, Python, etc. It is very simple to keep track of data in structures such as arrays and structures. This is not the case in prolog. In prolog information is stored in a dictionary of rules and facts. 

### a. Establishing Dynamic Facts
In the case of our chess AI, we keep track of a board position by asserting dynamic facts during run time. To enable runtime fact creation, we make the rule dynamic. 
```
:- dynamic(position/3).
```
Then, when we are ready to add a new position/3 fact, we do the following. 
```
asserta(position(Piece, Row, Col)).
```
This will add a new fact to the beginning of the chess_solver.pl which will be newly visible to rules further in the algorithm.

Note that Piece, Row, and Col are integers representing piece type and piece position respectively. We will cover the details of this implentation soon.

### b. Emulating Function Calls and Return Values 

We also emulate function calls and their respective return values by making rules which will intentionally be called with both unified input values as well as free variables which will be unified after the rule evaluation is completed. In this manner we can break up our minimax algorithm into human understandable syntax. Here's a quick example

```
functionFoo(InputX, OutputY) :-
  integer(InputX),
  OutputY is InputX + 1.
```

Calling the function is as simple as doing the following

```
functionFoo(4, Result).
```

Remember it is important that the variable hasn't been unified in this scope yet. If we decided to call 

```
write(Result).
```
the output would be 
```
5

yes.
```

Now that you hopefully have a better understanding of the mechanisms used in the solution, the next section will walk you through the actual code.

## 3. A Walkthrough 

After calling the compute method as described in the demo, minimax is called to search the Board for the appropriate ColorToMove at a depth of 3. The quote on quote "return value" of calling minimax is that the BestMove and Score will be unified with the Board and Score values which were deemed best for the ColorToMove.

```
minimax(Board, ColorToMove, Depth, BestMove, Score) :-
    /* Load our board and get the possible moves list */
    loadBoard(Board),
    switchColor(ColorToMove, NextColor),
    generateAllMoves(ColorToMove, PossibleMovesList),

    /* Unload our board now that we have all the information we need in lists */
    unloadBoard(),

    /* Create vector of new boards using different moves from PossibleMovesList */
    generateNewBoards(PossibleMovesList, Board, BoardsVec),

    /* Run minimax on all new boards and get list of [bestMove,score]. */
    DepthTemp is Depth - 1,
    getMinimaxResults(BoardsVec, NextColor, DepthTemp, ResultList),
    
    /* Set Score based on min or max of list and ColorToMove */
    (ColorToMove = black ->
        chooseMin(ResultList, BestMove, Score)
    ;
        chooseMax(ResultList, BestMove, Score)
    ).
```

Looking at this minimax rule, we can see that there are 5 main steps.

1. Load Board: is responsible for asserting new facts about the current board state as discussed in 2 a. 
2. a. Find new moves: calls the generateAllMoves "function" and unifies PossibleMovesList with a List of Moves. Moves is a List of Points. A Point is a pair of integers representing the row and column of the board. Ex. 
  ```
  PossibleMovesList = [[[1, 1],[2, 1]], ...].
  ```
b. Unload the board: Here we retract all position facts to make space for other minimax calls. This is just cleanup.
3. Generate new board states from moves: Now that we have a list of legal moves, we need to generate new board states so that we can recurse on them. generateNewBoards is a function that unifies a list of 2d Boards with a Move from the PossibleMovesList applied.
4. Run minimax on all new board states at depth - 1: Here is where program slows WAY down. (This should be obvious as we are now doing the actual recursion of minimax on all the possible next board states.) At this point, getMinimaxResults unifies the BestMove and Score of each minimax call on each of the new board states into a list of scores and their respective boards.  
5. Evaluate using minimax algorithm: Finally we have all the information we need. Here we call one of two minimax functions which simply unify the BestMove and Score of the minimax call to the best values found in the ResultList. 

At the top level minimax call, the now unified BestMove and Score together make up the information to decide which is the best move. Compute as we have already seen just spits this info to the user. 

## 4. Conclusions of Working in Prolog

The thing that I found most difficult in working with prolog was the difficulty of debugging. Mostly due to my ignorance, I was very inefficient in debugging problems that occurred during development. I would manually step through the entire solution using the trace and creep functionality. Only a day or two before the due date would I realise that there is a whole suite of helpful commands such as skip, leap, fail, and find, which allow immediate navigation through, in, and out of the prolog call stack. To any future readers, I would highly suggest reading up on the swi-prolog trace command.

# Results

## My Experience playing against this AI vs Modern AI. 

Searching 3 nodes deep is not very difficult, and most humans are able to reason possible opportunities given a move and an opponents reaction. Given this, I was able to easily beat this AI. However playing against Stockfish 11. It was the other way around. I am a decent chess player and am able to fairly consistently examine the best chess lines about 7-10 moves deep. However, Stockfish 11 with its superior ~30 node search depth is always able to set up and execute long and complicated tactics which by the time I see what is happening, is too late for me to recover.

I would be interested to see how games against Neural Net chess engines work, however, there is currently no human interface to test such AI. They can only be run against other engines programatically. 

## General Findings
1. Minimax is outdated
The search space for a minimax algorithm not implementing alpha-beta pruning is HUGE. If no pieces are captured, the size of a minimax tree is roughly 200^n states for a depth n. Without alpha beta pruning, the best my current AI can do is a minimax search with depth 3. If I try depth 4+, I immediately run more than 1G of RAM.
2. Stockfish 11 is incredible
The leading chess AI up until recently was an AI called Stockfish. This AI has been continuously worked on, and implements a diverse selection of algorithms. Some examples are its hyper optimized alpha-beta pruning minimax algorithm which (on-average) searches about 30 nodes deep as well as its plethera of manually written expert rules.
2. Neural Nets are the new best AI.
The new top dogs of the chess engine AI world are nerual net AI. The best chess engine currently is a Neural Net AI developed by Google called AlphaZero. It was run against Stockfish 8, and AlphaZero crushed it 28-72-0. Commentaries of AlphaZero chess almost always praise how well AlphaZero understands the game. It has an estimated ELO rating of somewhere between 3800-4200. For comparison, the world's best chess player Magnus Carlsen has an ELO of 2863.

# Final Thoughts on Ethical Implications of Project
I couldn't find a definition for the kind of bias that is present in my implementation, so I will do my best to explain. My goal was to create the best chess AI. By choosing an algorithm (given infinite time and memory) that is the best at finding the board state that wins the most piece points for a certain player, I have implicitely made the decision that winning by the most piece points is the best way to play chess. This is not true, however, and some of the best chess games are the ones that are ones that end early with only a few pieces taken. Just because a position wins a piece for black doesn't mean that in the long run that it was the best move. 

# Next Steps

I think a great next step would be to drop the outdated minimax algorithm and go for a neural net. Beyond that however, it would be very interesting to how specifically the models understand chess. Do they use the same strategy as Stockfish? or do they use some other strategy?
